In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('merged_data.csv')
result = data[data["name"].str.contains("kuba")]
result_train = data[data["name"].str.contains("kuba") == False]

result_train.drop('name', axis=1, inplace=True)
result.drop('name', axis=1, inplace=True)

X_train = result_train.iloc[:, :-1].values
labels_train = result_train.iloc[:, -1].values

X_test = result.iloc[:, :-1].values
labels_test = result.iloc[:, -1].values
#print(labels_train)
#print(labels_test)

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()

labels_train1 = encoder.fit_transform(labels_train)
print(labels_train1)

labels_test1 = encoder.transform(labels_test)
print(labels_test1)


[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [2]:
print('Input features shape', X_test.shape)
print('Actual labels shape', labels_test1.shape)

Input features shape (1269, 115)
Actual labels shape (1269, 8)


In [3]:
import numpy as np
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout


X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
# labels_train1 = np.array(labels_train1).reshape(labels_train1.shape[0], labels_train1.shape[1], 1)
# labels_test1 = np.array(labels_test1).reshape(labels_test1.shape[0], labels_test1.shape[1], 1)
#for n in range(len(X_train)):
#X_train=np.asarray(X_train).astype(np.int)
#X_train = X_train.ravel()
#X_test = np.asarray(X_test).ravel()
#for n in range(len(X_test)):
   # X_test[n]=np.asarray(X_test[n]).astype(np.int)

    
# [print(i.shape, i.dtype) for i in model.inputs]
# [print(o.shape, o.dtype) for o in model.outputs]
# [print(l.name, l.input_shape, l.dtype) for l in model.layers]
    
print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
print("Y Train shape: ", labels_train1.shape)
print("Y Test shape: ", labels_test1.shape)

X Train shape:  (3687, 115, 1)
X Test shape:  (1269, 115, 1)
Y Train shape:  (3687, 8)
Y Test shape:  (1269, 8)


In [12]:



# Create sequential model 
cnn_model = tf.keras.models.Sequential()

#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (X_train.shape[1],1)))

#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=64, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Third CNN layer with 128 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=128, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Fourth CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(3,), strides=2, padding='same'))
cnn_model.add(Dropout(0.5))

#Flatten the output
cnn_model.add(Flatten())

#Add a dense layer with 256 neurons
cnn_model.add(Dense(units = 256, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Add a dense layer with 512 neurons
cnn_model.add(Dense(units = 512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 8, activation='softmax'))

cnn_model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()


[print(i.shape, i.dtype) for i in cnn_model.inputs]
[print(o.shape, o.dtype) for o in cnn_model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in cnn_model.layers]

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 115, 32)           128       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 115, 64)           6208      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 115, 128)          24704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 58, 128)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 58, 128)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 7424)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

[None, None, None, None, None, None, None, None, None]

In [13]:

print(type(X_train), type(X_test), type(labels_train), type(labels_test))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [14]:
print("X Train shape: ", X_train.shape)
print("X Test shape: ", X_test.shape)
print("Y Train shape: ", labels_train1.shape)
print("Y Test shape: ", labels_test1.shape)

X Train shape:  (3687, 115, 1)
X Test shape:  (1269, 115, 1)
Y Train shape:  (3687, 8)
Y Test shape:  (1269, 8)


In [15]:

cnn_model_history = cnn_model.fit(X_train, labels_train1, epochs=10, batch_size =10, validation_data = (X_test, labels_test1))

Epoch 1/10
369/369 [==============================] - 3s 9ms/step - loss: 10349.2598 - accuracy: 0.4185 - val_loss: 933.1942 - val_accuracy: 0.2908
Epoch 2/10
369/369 [==============================] - 3s 8ms/step - loss: 147.1052 - accuracy: 0.5359 - val_loss: 114.8265 - val_accuracy: 0.3436
Epoch 3/10
369/369 [==============================] - 3s 8ms/step - loss: 35.0284 - accuracy: 0.5842 - val_loss: 119.0517 - val_accuracy: 0.2971
Epoch 4/10
369/369 [==============================] - 3s 8ms/step - loss: 22.4909 - accuracy: 0.6178 - val_loss: 14.1130 - val_accuracy: 0.4515
Epoch 5/10
369/369 [==============================] - 3s 8ms/step - loss: 21.9607 - accuracy: 0.6352 - val_loss: 15.5707 - val_accuracy: 0.3727
Epoch 6/10
369/369 [==============================] - 3s 8ms/step - loss: 11.3835 - accuracy: 0.6607 - val_loss: 17.1000 - val_accuracy: 0.5091
Epoch 7/10
369/369 [==============================] - 3s 8ms/step - loss: 15.2869 - accuracy: 0.6637 - val_loss: 18.6106 - val_ac

In [13]:
da = pd.read_csv('merged_data_normalized.csv')
print(da)

         mag_1     mag_2     mag_3     mag_4     mag_5     mag_6     mag_7  \
0     0.265739  0.256914  0.238871  0.204497  0.183374  0.212214  0.244017   
1     0.126638  0.063612  0.041273  0.050787  0.060859  0.076889  0.067438   
2     0.074331  0.042567  0.033469  0.047282  0.054843  0.064086  0.074341   
3     0.059915  0.051223  0.044134  0.036096  0.034211  0.040493  0.040402   
4     0.009380  0.005115  0.017383  0.015676  0.014560  0.037761  0.094945   
...        ...       ...       ...       ...       ...       ...       ...   
2157  0.287983  0.239614  0.209854  0.181102  0.164073  0.191248  0.221496   
2158  0.129129  0.117114  0.112617  0.108116  0.110345  0.141989  0.174905   
2159  0.297354  0.237052  0.185462  0.152254  0.140301  0.168356  0.198901   
2160  0.196470  0.169514  0.152267  0.134577  0.124661  0.147093  0.170851   
2161  0.292884  0.253282  0.222732  0.200584  0.200503  0.259909  0.322702   

         mag_8     mag_9    mag_10  ...   mag_493   mag_494   m